In [11]:
import h5py
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
import os

%matplotlib inline

POLLUTANT_MAP = {
    "1": "SO2", "5": "NO2", "7": "NO", "8": "NOx", "9": "CO",
    "10": "Benzene", "38": "PM10", "6001": "PM2.5", "6006": "O3"
}


In [ ]:
upload_output = widgets.Output()
station_dropdown = widgets.Dropdown(description="Station:", layout=widgets.Layout(width="50%"))
pollutant_dropdown = widgets.Dropdown(description="Pollutant:", layout=widgets.Layout(width="50%"))
plot_button = widgets.Button(description="📈 Plot", button_style="success")
output_plot = widgets.Output()
file_upload = widgets.FileUpload(accept=".h5", multiple=False, description="Upload .h5 File")


In [ ]:
def update_gui_from_file(h5_path):
    with h5py.File(h5_path, "r") as f:
        station_pollutant_map = {
            group: list(f["FR"][group].keys()) for group in f["FR"]
        }

    station_dropdown.options = list(station_pollutant_map.keys())

    def update_pollutants(*args):
        station = station_dropdown.value
        codes = station_pollutant_map.get(station, [])
        pollutant_dropdown.options = [(POLLUTANT_MAP.get(p, p), p) for p in codes]

    station_dropdown.observe(update_pollutants, names="value")
    update_pollutants()

    def on_plot(b):
        output_plot.clear_output()
        station = station_dropdown.value
        pollutant = pollutant_dropdown.value

        with h5py.File(h5_path, "r") as f:
            grp = f[f"FR/{station}/{pollutant}"]
            timestamps = [ts.decode("utf-8") for ts in grp["timestamps"][:]]
            values = grp["values"][:]
            unit = grp.attrs.get("unit", "")
            agg = grp.attrs.get("agg_type", "")
            val = grp.attrs.get("validity", "")

        df = pd.DataFrame({"timestamp": pd.to_datetime(timestamps), "value": values})
        label = POLLUTANT_MAP.get(pollutant, pollutant)

        with output_plot:
            plt.figure(figsize=(14, 5))
            plt.plot(df["timestamp"], df["value"], label=f"{label} ({unit})")
            plt.title(f"{station} | {label} | Agg: {agg} | Validity: {val}")
            plt.xlabel("Time")
            plt.ylabel(f"Concentration ({unit})")
            plt.grid(True)
            plt.legend()
            plt.tight_layout()
            plt.show()

    plot_button.on_click(on_plot)


In [ ]:
def on_file_upload(change):
    upload_output.clear_output()
    if not file_upload.value:
        with upload_output:
            print("Please upload a .h5 file.")
        return

    uploaded_file = file_upload.value[0]
    uploaded_filename = uploaded_file["name"]
    file_bytes = uploaded_file["content"]
    
    h5_path = "uploaded_voila_file.h5"
    with open(h5_path, "wb") as f:
        f.write(file_bytes)

    with upload_output:
        print(f"File uploaded: {uploaded_filename}")
    
    update_gui_from_file(h5_path)

    # if os.path.exists(h5_path):
    #     os.remove(h5_path)


In [15]:
file_upload.observe(on_file_upload, names="value")

ui = widgets.VBox([
    file_upload,
    upload_output,
    station_dropdown,
    pollutant_dropdown,
    plot_button,
    output_plot
])

display(ui)